
# Part 3- Feature Extraction

This is part 3 of our project where we are going to extract features from image files saved in parquet file from part 2

For feature extraction, we are using PIL, Image, IO libraries to carry out chip_data to byte array and assign those values in new column called as Features by using UDF function.


In [0]:
# Loading libraries

import import_ipynb
import SparkLauncher, HDFS
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.functions import lit
import sys
if not sys.warnoptions:
    import warnings
    
    warnings.simplefilter("ignore")
import os
import pyarrow as pa
from pyspark import SparkConf

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import venv_pack
from pyspark.sql.functions import array_contains
import io
from io import StringIO
import pyspark.sql.functions as f
import cv2
import tensorflow
import model
import cvlib as cv
import numpy as np

In [0]:
#Additional library
import io
import os
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input
from keras.applications.mobilenet import decode_predictions
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [0]:
# loading spark context
spark= SparkLauncher.get_spark_session()
hdfs = HDFS.get_hdfs()

Packing Virtual Environment: msitut1.tar.gz
Setting Environment Variables
Creating Spark Session: msitut1_data603_spark_session


In [0]:
sc=SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [0]:
hdfs.ls('/user/msitut1')

['/user/msitut1/.sparkStaging',
 '/user/msitut1/011d980ae9abea77_good_3.jpg',
 '/user/msitut1/012f947b95d444f5_good_2.jpg',
 '/user/msitut1/1411cbd5ae455abe_good_1.jpg',
 '/user/msitut1/1411cbd5ae455abe_good_5.jpg',
 '/user/msitut1/23bea7ad9d3a04f8_bad_2.jpg',
 '/user/msitut1/584286fa148884d7_bad_4.jpg',
 '/user/msitut1/6ca860c09a110014_good_4.jpg',
 '/user/msitut1/731d38b921b32d2c_bad_5.jpg',
 '/user/msitut1/a743ed1fcddfa0e8_bad_3.jpg',
 '/user/msitut1/c36a6b4b35aee1b8_bad_1.jpg',
 '/user/msitut1/df_data_no_data.parquet',
 '/user/msitut1/homework2_part1.parquet']

In [0]:
# Loading our saved parquet file
df_load_parquet_image = spark.read.parquet('/user/msitut1/df_data_no_data.parquet')

In [0]:
spark.catalog.clearCache()
sqlContext.clearCache()


In [0]:
df_load_parquet_image.printSchema()

root
 |-- ImageID: string (nullable = true)
 |-- LabelName: string (nullable = true)
 |-- Confidence_concat: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Good/Bad: string (nullable = true)
 |-- Target_Label: string (nullable = true)
 |-- Subset: string (nullable = true)
 |-- OriginalURL: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Confidence: string (nullable = true)
 |-- XMin: string (nullable = true)
 |-- XMax: string (nullable = true)
 |-- YMin: string (nullable = true)
 |-- YMax: string (nullable = true)
 |-- IsOccluded: string (nullable = true)
 |-- IsTruncated: string (nullable = true)
 |-- IsGroupOf: string (nullable = true)
 |-- IsDepiction: string (nullable = true)
 |-- IsInside: string (nullable = true)
 |-- chip_data: binary (nullable = true)



In [0]:
df_load_parquet_image.count()

3007

In [0]:
df_load_parquet_image.limit(5).toPandas()

,ImageID,LabelName,Confidence_concat,Good/Bad,Target_Label,Subset,OriginalURL,Source,Confidence,XMin,XMax,YMin,YMax,IsOccluded,IsTruncated,IsGroupOf,IsDepiction,IsInside,chip_data
0,001bf60cd96147ea,"/m/05r655, /m/0dzct","[1, 1]",Good Image,Female,train,https://c3.staticflickr.com/6/5537/12244508303...,activemil,1,0.281250,0.973750,0.000000,0.728464,-1,-1,-1,-1,-1,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,..."
1,001bf60cd96147ea,"/m/05r655, /m/0dzct","[1, 1]",Good Image,Female,train,https://c3.staticflickr.com/6/5537/12244508303...,xclick,1,0.451250,0.586875,0.402622,0.489700,0,0,0,0,0,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,..."
2,001bf60cd96147ea,"/m/05r655, /m/0dzct","[1, 1]",Good Image,Female,train,https://c3.staticflickr.com/6/5537/12244508303...,xclick,1,0.715000,0.855625,0.370787,0.473783,0,0,0,0,0,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,..."
3,001bf60cd96147ea,"/m/05r655, /m/0dzct","[1, 1]",Good Image,Female,train,https://c3.staticflickr.com/6/5537/12244508303...,xclick,1,0.571250,0.755000,0.787453,0.864232,0,0,0,0,0,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,..."
4,001bf60cd96147ea,"/m/05r655, /m/0dzct","[1, 1]",Good Image,Female,train,https://c3.staticflickr.com/6/5537/12244508303...,xclick,1,0.270000,0.960625,0.000000,0.999064,1,1,0,0,0,"[255, 216, 255, 224, 0, 16, 74, 70, 73, 70, 0,..."


In [0]:
#Preprocessing data and converting chip_data to byter array using udf
def load_and_preprocess(chip_data):
    import PIL.Image
    # Load the image
    image = load_img(io.BytesIO(chip_data), target_size = (224,224))
    # Save the image to a byte-buffer
    buff = io.BytesIO()
    image.save(buff, format = "JPEG")
    # Get the raw bytes of the jpeg data.
    byte_array = buff.getvalue()
    return byte_array
    
from array import array
from PIL import Image

def readimage(image_array):
        return Image.open(io.BytesIO(image_array))
#https://stackoverflow.com/questions/18491416/pil-convert-bytearray-to-image

In [0]:
# make a UDF
from pyspark.sql.types import *
from pyspark.sql.functions import udf
from collections import OrderedDict

udf_evaluate_chip = udf(load_and_preprocess, returnType = BinaryType())
df_load_parquet_image = df_load_parquet_image.withColumn("Features", udf_evaluate_chip("chip_data"))

In [0]:
temp=(df_load_parquet_image.select('Features').collect()[20].Features)

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 40530)
Traceback (most recent call last):
  File "/usr/lib64/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/scratch/data603_virtualenv/msitut1_2/lib64/python3.6/site-packages/pyspark/accumulators.py", line 267, in handle
    poll(authenticate_and_accum_updates)
  File "/scratch/data603_virtualenv/msitut1_2/lib64/python3.6/site-packages/pyspark/accumulators.py", line 239, in poll
    r, _, _ = select.select([self.rfile], [], [], 1)
ValueError: filedescriptor out of

## After resolving errors, our final data frame looks like below

In [0]:
df_load_parquet_image.count()

3007

In [0]:
df_load_parquet_image.printSchema()

root
 |-- ImageID: string (nullable = true)
 |-- LabelName: string (nullable = true)
 |-- Confidence_concat: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Good/Bad: string (nullable = true)
 |-- Target_Label: string (nullable = true)
 |-- Subset: string (nullable = true)
 |-- OriginalURL: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Confidence: string (nullable = true)
 |-- XMin: string (nullable = true)
 |-- XMax: string (nullable = true)
 |-- YMin: string (nullable = true)
 |-- YMax: string (nullable = true)
 |-- IsOccluded: string (nullable = true)
 |-- IsTruncated: string (nullable = true)
 |-- IsGroupOf: string (nullable = true)
 |-- IsDepiction: string (nullable = true)
 |-- IsInside: string (nullable = true)
 |-- chip_data: binary (nullable = true)
 |-- Features: binary (nullable = true)



In [0]:
df_load_parquet_image.write.parquet("/user/msitut1/project_part2.parquet")

In [0]:
#hdfs.delete('/user/msitut1/project_part2.parquet',recursive=True)
hdfs.ls('/user/msitut1')


['/user/msitut1/.sparkStaging',
 '/user/msitut1/011d980ae9abea77_good_3.jpg',
 '/user/msitut1/012f947b95d444f5_good_2.jpg',
 '/user/msitut1/1411cbd5ae455abe_good_1.jpg',
 '/user/msitut1/1411cbd5ae455abe_good_5.jpg',
 '/user/msitut1/23bea7ad9d3a04f8_bad_2.jpg',
 '/user/msitut1/584286fa148884d7_bad_4.jpg',
 '/user/msitut1/6ca860c09a110014_good_4.jpg',
 '/user/msitut1/731d38b921b32d2c_bad_5.jpg',
 '/user/msitut1/a743ed1fcddfa0e8_bad_3.jpg',
 '/user/msitut1/c36a6b4b35aee1b8_bad_1.jpg',
 '/user/msitut1/df_data_no_data.parquet',
 '/user/msitut1/homework2_part1.parquet',
 '/user/msitut1/project_part2.parquet']